In [10]:
import cv2
import numpy as np
import os

# List of Tesla models
models = ["Model_3", "Model_S", "Model_Y", "Model_X"]

# Load the pre-trained YOLO model
net = cv2.dnn.readNet("../yolov4/yolov4.weights", "../yolov4/yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels
with open("../yolov4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def detect_and_label_model(image_path, model_name):
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    # Track highest confidence detection
    max_confidence = 0
    best_box = None

    for detection in detections:
        for object_detection in detection:
            scores = object_detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Check if the detection is for a car and above confidence threshold
            if classes[class_id] == "car" and confidence > 0.5 and confidence > max_confidence:
                max_confidence = confidence
                center_x = int(object_detection[0] * width)
                center_y = int(object_detection[1] * height)
                w = int(object_detection[2] * width)
                h = int(object_detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                best_box = (x, y, w, h)

    # Draw the best bounding box if one was found
    if best_box:
        x, y, w, h = best_box
        label = f"{model_name}"
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    
    # Save the labeled image
    output_folder = f"dataset/{model_name}_labeled"
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, img)
    print(f"Labeled image saved to {output_path}")

# Process each model
for model in models:
    input_folder = f"dataset/{model}"
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.png')):
            detect_and_label_model(os.path.join(input_folder, filename), model)

Labeled image saved to dataset/Model_3_labeled/img36.jpg
Labeled image saved to dataset/Model_3_labeled/img37.jpg
Labeled image saved to dataset/Model_3_labeled/img35.jpg
Labeled image saved to dataset/Model_3_labeled/img30.jpg
Labeled image saved to dataset/Model_3_labeled/img31.jpg
Labeled image saved to dataset/Model_3_labeled/img33.jpg
Labeled image saved to dataset/Model_3_labeled/img32.jpg
Labeled image saved to dataset/Model_3_labeled/img41.jpg
Labeled image saved to dataset/Model_3_labeled/img40.jpg
Labeled image saved to dataset/Model_3_labeled/img42.jpg
Labeled image saved to dataset/Model_3_labeled/img44.jpg
Labeled image saved to dataset/Model_3_labeled/img9.jpg
Labeled image saved to dataset/Model_3_labeled/img8.jpg
Labeled image saved to dataset/Model_3_labeled/img45.jpg
Labeled image saved to dataset/Model_3_labeled/img5.jpg


KeyboardInterrupt: 